# Compute K-LL Auger rates for neon-like iron in a Debye-Hückel potential

In [6]:
using JAC

A plasma environment generally affects both, the level structure of atoms and ions as well their rates and cross sections. In this tutorial, we wish to compute the K-LL rates for neon-like iron in a Debye-Hückel potential following the work by Deprince *et al* (2019). 

A Debye-Hückel potential may affect the Auger rates of an atom or ion in three different ways: (i) the level structure due to the screened electron-nucleus and electron-electron interaction; (ii) the shape of the potential, seen by the emitted electrons, due to the plasma screening of the ionic core; as well as (iii) the screening of the electron-electron interaction within the Auger matrix. In the present implementation, only the (screened) Coulomb interaction can be taken into account for the computation of the plasma-modified Auger rates, i.e. **no** Breit interactions nor QED estimates are supported by the JAC program.

As with most other (atomic) processes, the compuation of plasma-modified Auger is supported in JAC by a special process identifier `AugerInPlasma` and the corresponding settings `PlasmaShift.AugerSettings`. These settings can be used to define the screening parameter $\lambda$. 


Let us start again from the `PlasmaShift.AugerSettings` to understand what and how we can control the computations:

In [2]:
? PlasmaShift.AugerSettings

`struct  PlasmaShift.AugerSettings  <:  AbstractProcessSettings`       ... defines a type for the details and parameters of computing Auger rates with plasma interactions.

```
+ plasmaModel         ::AbstractPlasmaModel    ... Specify a particular plasma model, e.g. ion-sphere, debye.
+ lambdaDebye         ::Float64                ... The lambda parameter of different plasma models.
+ ionSphereR0         ::Float64                ... The effective radius of the ion-sphere model.
+ NoBoundElectrons    ::Int64                  ... Effective number of bound electrons.
+ printBefore         ::Bool                   ... True, if all energies and lines are printed before their 
                                                   evaluation.
+ lineSelection       ::LineSelection          ... Specifies the selected levels, if any.
```

---

`PlasmaShift.AugerSettings()`  ... constructor for a standard instance of PlasmaShift.AugerSettings.


In [3]:
PlasmaShift.AugerSettings()   # show the default values

plasmaModel:             Debye-Hueckel model  
lambdaDebye:             0.25  
ionSphereR0:             0.0  
NoBoundElectrons:        0  
printBefore:             true  
lineSelection:           Inactive LineSelection.  


We can re-define these settings by specifying the screening parameter and the selected lines to be 
calculated:

In [4]:
pSettings = PlasmaShift.AugerSettings(PlasmaShift.DebyeHueckel(), 0.1, 0., 0, true, 
                                      LineSelection(true, indexPairs=[(1,1), (1,2)]) )

plasmaModel:             Debye-Hueckel model  
lambdaDebye:             0.1  
ionSphereR0:             0.0  
NoBoundElectrons:        0  
printBefore:             true  
lineSelection:           LineSelection:  indexPairs = [(1, 1), (1, 2)];    symmetryPairs = Tuple{LevelSymmetry, LevelSymmetry}[].  



As usual, we can now defined and run the desired computation by *defining* and *performing* an 
`Atomic.Computation()`:


In [5]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-5, h = 5.0e-2, hp = 1.3e-2, rbox = 10.)
wa   = Atomic.Computation(Atomic.Computation(), name="K-LL Auger spectrum in a Debye-Hueckel potential",  
              grid=grid, nuclearModel=Nuclear.Model(26.), 
              initialConfigs=[Configuration("1s 2s^2 2p^6")],
              finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                              Configuration("1s^2 2p^6")], processSettings = pSettings)
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -3.41095879e+02    -3.41097837e+02    +5.74130819e-06    
      2      2s_1/2      -8.54687057e+01    -8.54689584e+01    +2.95668826e-06    
      3      3s_1/2      -3.78993728e+01    -3.78994478e+01    +1.97920450e-06    
      4      4s_1/2      -2.12819069e+01    -


Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.8438118e+02;   self-cons'cy = 9.0674e-02  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -5.1573110e+01;   self-cons'cy = 2.4734e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -4.8296776e+01;   self-cons'cy = 2.7789e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -4.8395506e+01;   self-cons'cy = 2.7272e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.9025088e+02;   self-cons'cy = 1.0215e-02  [6.8942e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -5.3407305e+01;   self-cons'cy = 1.7472e-02  [6.8942e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -5.0035914e+01;   self-cons'cy = 1.7686e-02  [4.6775e-02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -4.9402033e+01;   self-cons'cy = 1.0292e-02  [2.6246e-02 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.8958783e+02;   self-

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -8.54689566e+01    -8.54689584e+01    +2.03867573e-08    
      2      3p_1/2      -3.78994472e+01    -3.78994478e+01    +1.61654735e-08    
      3      4p_1/2      -2.12819382e+01    -2.12819385e+01    +1.27951885e-08    
      4      5p_1/2      -1.36039712e+01    -1.36039714e+01    +1.04817010e-08    
      5      6p_1/2      -9.43887313e+00    -9.43887322e+00    +8.84543065e-09    
      6      7p_1/2      -6.93005919e+00    -6.93005925e+00    +7.63709558e-09    
      7      8p_1/2      -5.30306675e+00    -5.30306679e+00    +7.13386445e-09    
      8      9p_1/2      -4.18832685e+00    -4.18833237e+00    +1.31799103e-06    
      9     10p_1/2      -3.39009309e+00    -3.39139342e+00    +3.83566403e-04    
     10    

Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.8171031e+02;   self-cons'cy = 9.5352e-02  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -5.4263186e+01;   self-cons'cy = 2.2332e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -5.0970415e+01;   self-cons'cy = 2.5285e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -5.0960454e+01;   self-cons'cy = 2.4866e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.8652171e+02;   self-cons'cy = 8.4673e-03  [7.7451e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -5.5679853e+01;   self-cons'cy = 1.2885e-02  [7.7451e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -5.2283326e+01;   self-cons'cy = 1.2715e-02  [9.8110e-02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -5.1716536e+01;   self-cons'cy = 7.3637e-03  [6.0361e-02 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.8612128e+02;   self-c

┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:281


...

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=1.8882e+02,  kappa=2 [mpt=836, r[mtp]=7.5058e+00, smallest eigenvalue=1.0900e-10].
>> Pure-sine normalized continuum orbital with normalization constant N=3.1809e+00 and phase phi=1.9230e+00 at r=7.5058e+00 a.u.


┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:285
┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:281


>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=1.8848e+02,  kappa=-1 [mpt=836, r[mtp]=7.5058e+00, smallest eigenvalue=1.0517e-07].
>> Pure-sine normalized continuum orbital with normalization constant N=3.1857e+00 and phase phi=3.5327e-01 at r=7.5058e+00 a.u.


┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:285


 
  Auger rates (without angular parameters): 

  ----------------------------------------------------------------------------------------------------------
       i-level-f           i--J^P--f           Energy      Electron energy    Auger rate        alpha_2      
                                                [eV]             [eV]            [1/s]                        
  ----------------------------------------------------------------------------------------------------------
         1 --    1       1/2 + --> 2 +      -2.300919e+04    5.138055e+03    5.195110e+17       0.0000e+00    
         1 --    2       1/2 + --> 0 +      -2.300919e+04    5.128933e+03    2.597555e+17       0.0000e+00    
  ----------------------------------------------------------------------------------------------------------
 
  Auger lifetimes, total rates and widths:
 
  --------------------------------------------------------------------------------------------------------
     Level       J^P        


Similar computation will be made available also for photoionization cross sections in the near future.
